RNN

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
os.chdir("/content/drive/Othercomputers/My Laptop/nlp_scratch")

In [3]:
import torch
from torch import nn
import torch.functional as F
import numpy as np

In [4]:
with open("data/names.txt") as f:
    names_list = f.readlines()

In [5]:
chars = sorted(set("".join(names_list)))
vocab_size=len(chars)

In [6]:
char_to_idx = {ch:i for i, ch in enumerate(chars)}
idx_to_char = {i:ch for i,ch in enumerate(chars)}

In [7]:
class NameDataset(torch.utils.data.Dataset):

    def __init__(self,names):
        super().__init__()
        self.names=names

    def __getitem__(self, index):
        return self.names[index]

    def __len__(self):
        return len(self.names)

In [8]:
# Dataloader
dataloader = torch.utils.data.DataLoader(dataset=NameDataset(names=names_list),batch_size=256,shuffle=True)

In [9]:
# multi-layer RNN
from models.attention.rnn import RNNv2

In [73]:
rnn = RNNv2(vocab_size,32,num_layers=1)
# rnn.load_state_dict(torch.load("rnn_l2_weights.pth"))
learning_rate = 0.001
criterion = nn.CrossEntropyLoss(reduction="mean")
optim = torch.optim.Adam(rnn.parameters(),learning_rate)
onehot_tensor = torch.eye(27)
softmax= nn.Softmax(dim=-1)

In [74]:
# training
torch.autograd.set_detect_anomaly(True)
low_loss = 0
n_epochs = 10
for j in range(n_epochs):
    total_loss= 0
    for names in dataloader:
        batch_loss =0
        for name in names:
            input_idx = torch.tensor([char_to_idx[ch] for ch in name[:-1]])
            target_idx = torch.tensor([char_to_idx[ch] for ch in name[1:]])
            input_onehot = onehot_tensor[input_idx]
            rnn.zero_grad()
            logits,_ = rnn(input_onehot)
            # softmax_output = softmax(logits)
            loss = criterion(logits,target_idx)
            loss.backward()
            optim.step()
            # batch_loss+=loss.item()
            total_loss+=loss.item()
        # print(f"Batch , Loss: {batch_loss / len(names):.4f}")
    # save the weights
    # torch.save(rnn.state_dict(),"rnn_l2_weights.pth")
    print(f"Epoch {j+1}, Loss: {total_loss / len(names_list):.4f}")

Epoch 1, Loss: 3.1280
Epoch 2, Loss: 2.3690
Epoch 3, Loss: 2.2778
Epoch 4, Loss: 2.2193
Epoch 5, Loss: 2.1864
Epoch 6, Loss: 2.1650
Epoch 7, Loss: 2.1487
Epoch 8, Loss: 2.1362
Epoch 9, Loss: 2.1264
Epoch 10, Loss: 2.1189


In [ ]:
# save the weights
torch.save(rnn.state_dict(),"rnn_l1_weights.pth")

# load weights
# rnn.load_state_dict(torch.load("rnn_l1_weights.pth"))

In [21]:
input.expand(1,-1)

tensor([[0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [71]:
# sampling
start = np.random.choice(chars)
temp = 0.01
print(start)
with torch.no_grad():
    input = onehot_tensor[char_to_idx[start]]
    output_name = start
    for _ in range(20):
        logits,_ = rnn(input.expand(1,-1))
        output = softmax(logits)
        idx = torch.multinomial(output,1)
        idx = idx.item()
        if idx_to_char[idx]=="\n":
            break
        else:
            letter = idx_to_char[idx]
            output_name+=letter
            input = onehot_tensor[char_to_idx[letter]]


m


In [72]:
output_name

'mamisosolulicarinarel'

### LSTM

### Seq2Seq

In [ ]:
import torch
